# Analysis

## Extract "flu-related" words

In [2]:
from gensim.models import KeyedVectors

kv_fname = 'kowiki-neg-300.kv'
w2v_kv = KeyedVectors.load(f'./models/{kv_fname}')

In [4]:
w2v_kv.most_similar_cosmul(positive=['독감', '증상'], topn=30)

[('질병', 0.6456781029701233),
 ('합병증', 0.6416015028953552),
 ('패혈증', 0.6396576166152954),
 ('피부병', 0.6389033198356628),
 ('황달', 0.637260377407074),
 ('매독', 0.6342878341674805),
 ('기관지염', 0.6325851678848267),
 ('폐렴', 0.6323192715644836),
 ('급성', 0.6281660795211792),
 ('복통', 0.6278930902481079),
 ('홍역', 0.6278191804885864),
 ('대상포진', 0.6273614764213562),
 ('뎅기열', 0.6261769533157349),
 ('천식', 0.6254749298095703),
 ('뇌염', 0.6234469413757324),
 ('무증상', 0.6224794387817383),
 ('관절통', 0.6186830997467041),
 ('골다공증', 0.6164602041244507),
 ('파킨슨병', 0.615439772605896),
 ('폐질환', 0.6129343509674072),
 ('신장염', 0.6128467917442322),
 ('말라리아', 0.612651526927948),
 ('발병', 0.6126167178153992),
 ('심근염', 0.6122466325759888),
 ('결핵', 0.6121384501457214),
 ('부작용', 0.6108651161193848),
 ('뇌전증', 0.6089601516723633),
 ('알츠하이머병', 0.6067097783088684),
 ('파상풍', 0.6065411567687988),
 ('가려움증', 0.6062012314796448)]

In [11]:
import urllib
import json
from tqdm import tqdm

client_id = 'B4Vmp4Wt5ECl_fNWgIXR'
client_secret = 'WnmtyQRoNa'
naver_trends_url = 'https://openapi.naver.com/v1/datalab/search'

flu_related_words = [w for w, _ in w2v_kv.most_similar_cosmul(positive=['독감', '증상'], topn=1000)]
word_trends = {}

for word in tqdm(flu_related_words):
    try:
        body = ('{"startDate\":"2017-01-02","endDate":"2021-06-06","timeUnit":"week",' 
                + '"keywordGroups":[{"groupName":"' + word + '","keywords":[\"' + word + '\"]}]}')
        request = urllib.request.Request(naver_trends_url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if rescode==200:
            resbody = response.read()
            resbody = json.loads(resbody.decode('utf8'))
            word_trends[word] = resbody['results'][0]['data']
        else:
            print(f'request is failed for {word}, error code: {rescode}')
    except:
        print(f'request is failed for {word}, error code: {rescode}')
        continue

100%|██████████| 999/999 [01:32<00:00, 10.76it/s]


In [63]:
from datetime import datetime
import pandas as pd

word_trends_fz = {}
for word, data in word_trends.items():
    word_trends_fz[word] = []
    i = 0
    for date in pd.date_range('20170102', '20210531', freq='W-MON').strftime('%Y-%m-%d'):
        if len(data)==0:
            word_trends_fz[word].append({'period': date, 'ratio': 0.0})
        else:
            if data[i]['period'] == date:
                word_trends_fz[word].append({'period': date, 'ratio': data[i]['ratio']})
                if i+1 < len(data):
                    i += 1
            else:
                word_trends_fz[word].append({'period': date, 'ratio': 0.0})

In [66]:
word_trends_df = pd.DataFrame({word: [prdict['ratio'] for prdict in prlist] for word, prlist in word_trends_fz.items()})
word_trends_df.insert(0, 'period', pd.date_range('20170102', '20210531', freq='W-MON').strftime('%Y-%m-%d'))
word_trends_df.to_csv('./data/word_trends.csv', index=False)

In [68]:
word_trends_df = pd.read_csv('./data/word_trends.csv')
word_trends_df

,period,질병,합병증,패혈증,피부병,황달,매독,기관지염,폐렴,급성,...,성욕,뇌막,뇌일혈,에리트로포이에틴,원추각막,열대병,요법,마비저,손상,통풍
0,2017-01-02,34.47181,41.12814,0.50525,42.93985,9.26411,5.46836,64.21550,5.74694,24.46393,...,100.00000,23.92473,18.26252,5.33980,100.00000,15.32258,38.62068,0.0,38.85653,61.21533
1,2017-01-09,36.39417,37.18510,0.86922,42.97201,9.21393,5.98957,58.60113,4.81950,20.66276,...,96.28552,23.65591,14.33100,5.33980,96.13677,4.03225,57.93103,0.0,40.81524,4.68184
2,2017-01-16,31.46696,37.89704,0.60639,43.61531,22.23031,5.82590,51.64461,4.45071,25.43859,...,79.06976,23.38709,15.53582,5.33980,90.10702,15.32258,46.20689,0.0,38.16834,3.69813
3,2017-01-23,23.81485,31.98247,0.39328,42.87552,9.12975,4.99098,37.98676,3.79676,20.27290,...,73.02971,11.55913,11.73113,10.67961,85.82615,19.35483,42.75862,0.0,28.69242,3.34324
4,2017-01-30,30.64576,31.92771,0.43524,46.47796,9.25764,5.56968,41.55009,4.04877,36.15984,...,88.27519,26.61290,11.73113,9.22330,91.28164,17.74193,28.27586,0.0,34.88618,3.41878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,2021-05-03,53.45278,64.62212,0.50041,28.43357,8.79144,6.01782,22.93950,2.76159,26.41325,...,48.38501,57.25806,15.53582,28.64077,19.05507,20.16129,48.27586,0.0,40.86818,3.84868
227,2021-05-10,77.92086,66.86746,0.48556,30.74943,8.92255,6.28087,26.28544,2.94768,32.84600,...,54.45736,69.62365,17.31135,13.10679,18.74184,16.93548,73.10344,0.0,49.33827,3.97368
228,2021-05-17,67.95446,58.37897,0.49975,29.81666,9.53606,6.11330,24.27221,2.78343,28.55750,...,49.61240,66.39784,17.81864,10.19417,16.83633,14.51612,81.37931,0.0,42.56220,4.43700
229,2021-05-24,74.80403,62.32201,0.59264,29.72016,9.69794,6.33640,24.95274,2.91114,33.43079,...,50.35529,69.35483,17.94546,21.84466,21.63925,34.67741,84.13793,0.0,44.73266,4.34107


## Make wor2vec

In [4]:
from gensim.models import Word2Vec

min_count = 5
window_size = 5
num_neg = 15
vector_size = 300

print('Making sentences as list...')
sents = []
corpus_fname = 'kowiki_corpus.txt'

with open(f'./data/{corpus_fname}', 'r', encoding='utf8') as fin:
    line = fin.readline()
    while line:
        words = line.split()
        sents.append(words)
        line = fin.readline()

print('Making word vectors...')
w2v_model = Word2Vec(sents, vector_size=vector_size, min_count=min_count, negative=num_neg, window=window_size)

w2v_model.save('./models/kowiki-neg-300.bin')

Making sentences as list...
Making word vectors...


In [13]:
w2v_model.wv.most_similar('독감')

[('인플루엔자', 0.7366062998771667),
 ('홍역', 0.6661979556083679),
 ('콜레라', 0.652269721031189),
 ('출혈열', 0.648910641670227),
 ('유행병', 0.6479013562202454),
 ('전염병', 0.6393334269523621),
 ('뎅기열', 0.6355220079421997),
 ('대유행', 0.6307356357574463),
 ('말라리아', 0.6306702494621277),
 ('광견병', 0.6233310103416443)]

In [14]:
w2v_kv = w2v_model.wv
w2v_kv.save('./models/kowiki-neg-300.kv')

## Build corpus

In [4]:
from xml.etree import ElementTree as ET

wiki_fname = 'kowiki-latest-pages-articles.xml'

In [5]:
import re

def clean_text(text):    
    # Common
    text = re.sub("(?s)<ref>.+?</ref>", "", text) # remove reference links
    text = re.sub("(?s)<[^>]+>", "", text) # remove html tags
    text = re.sub("&[a-z]+;", "", text) # remove html entities
    text = re.sub("(?s){{.+?}}", "", text) # remove markup tags
    text = re.sub("(?s){.+?}", "", text) # remove markup tags
    text = re.sub("(?s)\[\[([^]]+\|)", "", text) # remove link target strings
    text = re.sub("(?s)\[\[([^]]+\:.+?]])", "", text) # remove media links
    
    text = re.sub("[']{5}", "", text) # remove italic+bold symbols
    text = re.sub("[']{3}", "", text) # remove bold symbols
    text = re.sub("[']{2}", "", text) # remove italic symbols
    
    text = re.sub(u"[^\s\r\n가-힣.?!]", " ", text) # Replace unacceptable characters with a space.
    text = re.sub('([.?!]){2,}', '\\1', text) # remove repeated punctuation
    text = re.sub('\s[.?!]\s', '', text) # remove isolated punctuation
    
    # Common
    text = re.sub("\s{2,}", " ", text) # Squeeze spaces.
    return text

In [10]:
def sentence_segment(text):
    '''
    Args:
      text: A string. A unsegmented paragraph.
    
    Returns:
      A list of sentences.
    '''
    return re.split('([.?!])?[\n]+|[.?!] ', text)

In [11]:
from konlpy.tag import Mecab

mecab = Mecab()
def word_segment(text):
    return [word for word, _ in mecab.pos(text)]

In [47]:
with open(f'./data/{wiki_fname.split("-")[0]}_corpus.txt', 'w', encoding='utf-8') as fout:
    fout.close()

In [49]:
import itertools
from tqdm import tqdm

ns = '{http://www.mediawiki.org/xml/export-0.10/}'
with open(f'./data/{wiki_fname.split("-")[0]}_corpus.txt', 'w', encoding='utf-8') as fout:
    for _, elem in tqdm(ET.iterparse(f'./data/{wiki_fname}')):
        try:
            tag = elem.tag.replace(ns, '')
            if tag == 'text':
                running_text = clean_text(elem.text)
                sents = sentence_segment(running_text)
                for sent in sents:
                    if sent:
                        words = word_segment(sent)
                        if len(words) > 10:
                            fout.write(' '.join(words) + '\n')
        except:
            continue
        elem.clear()

25979812it [30:56, 13991.10it/s]
